# Initialize

In [1]:
# turn on autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os,sys
ROOT = os.path.abspath("..")          # if your notebook/ lives under project-root/notebook/
SRC  = os.path.join(ROOT, "src")
sys.path.append(SRC)

# Options Underlying

In [3]:
import pandas as pd

options_df = pd.DataFrame({
    'mult':[100,100],
    'Expiration': ['1/15/27','1/15/27'],
    'Strike':[250,250],
    'Spot':[252.29,252.29],
    'CP':['P','C'],
    'EA':['A','A'],
    'Texp':[1.2416,1.2416],
    'Vol':['27.45%','27.45%'],
    'Borrow':[-0.006,-0.0060]
})

dividends_df = pd.DataFrame({
    'Date':['11/8/25','2/8/26','5/10/26','8/9/26','11/8/26','2/8/27','5/8/27','8/8/27'],
    'Amount':[0.26,0.26,0.26,0.26,0.26,0.40,0.40,0.40]
})

str_underlying='AAPL'
pd_val_date=pd.Timestamp('10/17/2025') 

In [4]:
from listed_pricer_data_loader import get_ivol_yc, apply_ivol_yc, fetch_and_pick
from listed_pricer_qlib import price_table_with_divs_ql

yc_results=get_ivol_yc(pd_val_date)

options_df2=apply_ivol_yc(yc_results,options_df)
options_df2

priced_ql_ir=price_table_with_divs_ql(
    options_df2, dividends_df,
    #valuation_date='10/17/2025',
    valuation_date=pd_val_date.strftime('%m/%d/%Y'),
    default_r=0.0308,                  # use your carry here; bump if needed to match combo
    init_vol=0.2755,           # optional: seed for the SimpleQuote
    t_grid=400, x_grid=200,
    want_greeks=True,
    compute_combo=True
)


# legs_df with your desired contracts:
# columns: Expiration (any parseable date), Strike (float), CP ('C'/'P')
legs_df=options_df2[['Expiration','Strike','CP']]
out = fetch_and_pick(str_underlying, trade_date=pd_val_date.strftime('%Y-%m-%d'), legs_df=legs_df, snap_to_nearest=False)
print(out["issues"])     # any misses or snaps
print(out["matched_df"]) # exact rows you can pass to your pric
out_relevant=out["matched_df"][['expiration','strike','cp','bid','ask','iv','openInterest','volume','delta','gamma','vega','underlying_price','optionId']]
df_combine=pd.concat([priced_ql_ir,out_relevant],axis=1)
df_combine
df_show=df_combine[["Expiration" ,"Strike","Spot","CP", "EA","Texp","Vol","Borrow", "bid","Theo","ask"]]
df_show

[]
  symbol cp  expiration  strike  snapped_strike   bid    ask    mark  \
0   AAPL  P  2027-01-15   250.0           False  24.6  25.00  24.800   
1   AAPL  C  2027-01-15   250.0           False  36.7  37.65  37.175   

         iv  openInterest  volume     delta     gamma      vega  \
0  0.267427          5031      57 -0.401476  0.005546  1.071017   
1  0.287485          9767      57  0.618236  0.004756  1.068378   

   underlying_price   optionId  
0            252.29  134511704  
1            252.29  134511703  


,Expiration,Strike,Spot,CP,EA,Texp,Vol,Borrow,bid,Theo,ask
0,1/15/27,250,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.854431,25.00
1,1/15/27,250,252.29,C,A,1.2416,27.45%,-0.006,36.7,37.275696,37.65


In [10]:

priced_ql = price_table_with_divs_ql(
    options_df, dividends_df,
    #valuation_date='10/17/2025',
    valuation_date=pd_val_date.strftime('%m/%d/%Y'),
    default_r=0.0308,                  # use your carry here; bump if needed to match combo
    init_vol=0.2755,           # optional: seed for the SimpleQuote
    t_grid=400, x_grid=200,
    want_greeks=True,
    compute_combo=True
)

print(priced_ql)

   mult Expiration  Strike    Spot CP EA    Texp     Vol  Borrow       Theo  \
0   100    1/15/27     250  252.29  P  A  1.2416  27.45%  -0.006  25.293813   
1   100    1/15/27     250  252.29  C  A  1.2416  27.45%  -0.006  36.685118   

      Delta     Gamma      Theta      Vega  Combo_C_minus_P  
0 -0.399344  0.005402  -8.473548  1.074258        11.391305  
1  0.628669  0.004961 -16.608745  1.074383        11.391305  
